In [227]:
import csv
import pdb
import numpy as np
import random
random.seed(1000)
import ast
import os
import collections
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import cm
from matplotlib.colors import LightSource
import scipy.stats

In [542]:
os.getcwd()

'/Users/subhrasishchakraborty/Desktop/lumbar-conformal-main/scripts'

In [543]:
os.chdir('/Users/subhrasishchakraborty/Desktop/lumbar-conformal-main/scripts')
import conformal_demo

In [544]:
num_trials = 10
alphas = np.array([0.01, 0.05, 0.10, 0.15, 0.2])
cal_percent = 0.05

In [545]:
#df_cal = pd.read_csv('/Users/subhrasishchakraborty/Desktop/calibration_data.csv')
#df_val = pd.read_csv('/Users/subhrasishchakraborty/Desktop/validation_data.csv')
#df_cal = df_cal.values
#df_val = df_val.values
df = pd.read_csv('/Users/subhrasishchakraborty/Desktop/data.csv')

In [546]:
df.head()

,Unnamed: 0,X3,X2,X4,X1,cal_labels
0,9,0.058987,0.201998,0.021008,0.718008,1
1,58,0.164995,0.065472,0.750588,0.018944,4
2,90,0.309678,0.413343,0.159028,0.117951,2
3,110,0.128789,0.305809,0.023600,0.541802,2
4,123,0.291153,0.285966,0.166259,0.256622,2


In [547]:
df.shape

(2000, 6)

In [548]:
def class_conditional_eval(sets, labels, classes=['neg', 'mod', 'mil', 'sev']):
    res = collections.defaultdict()
    for i, v in enumerate(classes):
        class_cond_sets = sets[labels == i]
        n = class_cond_sets.shape[0]
        res[v] = {
            'count': n,
            'coverage': (class_cond_sets[np.arange(n), i].sum() / n) if n > 0 else 0,
            'size': class_cond_sets.sum(1).mean(),
        }
    return res

def size_conditional_eval(sets, labels, sizes=[1, 2, 3, 4]):
    res = collections.defaultdict()
    for i, v in enumerate(sizes):
        size_cond_sets = sets[sets.sum(1) == v]
        n = size_cond_sets.shape[0]
        res[v] = {
            'count': n,
            'coverage': (size_cond_sets[np.arange(n), i].sum() / n) if n > 0 else 0,
        }
    return res

In [549]:
# alpha_lac_overall_coverage = collections.defaultdict(list)
# alpha_cdf_overall_coverage = collections.defaultdict(list)
alpha_aps_overall_coverage = collections.defaultdict(list)
# alpha_lac_overall_size = collections.defaultdict(list)
# alpha_cdf_overall_size = collections.defaultdict(list)
alpha_aps_overall_size = collections.defaultdict(list)

# alpha_lac_class_cond_count = collections.defaultdict(list)
# alpha_cdf_class_cond_count = collections.defaultdict(list)
alpha_aps_class_cond_count = collections.defaultdict(list)
# alpha_lac_class_cond_coverage = collections.defaultdict(list)
# alpha_cdf_class_cond_coverage = collections.defaultdict(list)
alpha_aps_class_cond_coverage = collections.defaultdict(list)
# alpha_lac_class_cond_size = collections.defaultdict(list)
# alpha_cdf_class_cond_size = collections.defaultdict(list)
alpha_aps_class_cond_size = collections.defaultdict(list)
# alpha_lac_size_cond_count = collections.defaultdict(list)
# alpha_cdf_size_cond_count = collections.defaultdict(list)
alpha_aps_size_cond_count = collections.defaultdict(list)
# alpha_lac_size_cond_coverage = collections.defaultdict(list)
# alpha_cdf_size_cond_coverage = collections.defaultdict(list)
alpha_aps_size_cond_coverage = collections.defaultdict(list)

In [550]:
for alpha in alphas:
    #lac_overall_coverage = []
    #cdf_overall_coverage = []
    aps_overall_coverage = []
    #lac_overall_size = []
    #cdf_overall_size = []
    aps_overall_size = []

#     lac_class_cond_count = collections.defaultdict(list)
#     cdf_class_cond_count = collections.defaultdict(list)
    aps_class_cond_count = collections.defaultdict(list)
#     lac_class_cond_coverage = collections.defaultdict(list)
#     cdf_class_cond_coverage = collections.defaultdict(list)
    aps_class_cond_coverage = collections.defaultdict(list)
#     lac_class_cond_size = collections.defaultdict(list)
#     cdf_class_cond_size = collections.defaultdict(list)
    aps_class_cond_size = collections.defaultdict(list)
#     lac_size_cond_count = collections.defaultdict(list)
#     cdf_size_cond_count = collections.defaultdict(list)
    aps_size_cond_count = collections.defaultdict(list)
#     lac_size_cond_coverage = collections.defaultdict(list)
#     cdf_size_cond_coverage = collections.defaultdict(list)
    aps_size_cond_coverage = collections.defaultdict(list)
#     lac_size_cond_size = collections.defaultdict(list)
#     cdf_size_cond_size = collections.defaultdict(list)
    aps_size_cond_size = collections.defaultdict(list)
    for trial in range(num_trials):
        
        df_cal = df.sample(frac=cal_percent)
        df_val = df.drop(df_cal.index)
        df_cal = df_cal.values
        df_val = df_val.values
        cal_scores = df_cal[:,1:5]
        val_scores = df_val[:,1:5]
        cal_labels = df_cal[:,5]-1
        val_labels = df_val[:,5]-1
        cal_labels = cal_labels.astype('int32')
        val_labels = val_labels.astype('int32')
        
        
        
        # Naive LAC
#         lac_qhat = np.quantile(
#             conformal_demo.lac_score_function(
#                 np.copy(cal_scores),
#                 np.copy(cal_labels)
#             ),
#             np.ceil((cal_scores.shape[0] + 1)*(1 - alpha))/cal_scores.shape[0],
#             interpolation='higher',
#         )
#         lac_pred_sets = conformal_demo.lac_prediction(np.copy(val_scores), lac_qhat)
#         lac_overall_coverage.append(lac_pred_sets[np.arange(val_scores.shape[0]), val_labels].sum() / val_labels.shape[0])
#         lac_overall_size.append(lac_pred_sets.sum(1).mean())
#         for _class, cond_res in class_conditional_eval(lac_pred_sets, val_labels).items():
#             lac_class_cond_count[_class].append(cond_res['count'])
#             lac_class_cond_coverage[_class].append(cond_res['coverage'])
#             lac_class_cond_size[_class].append(cond_res['size'])
#         for _size, cond_res in size_conditional_eval(lac_pred_sets, val_labels).items():
#             lac_size_cond_count[_size].append(cond_res['count'])
#             lac_size_cond_coverage[_size].append(cond_res['coverage'])
        
        # Ordinal CDF
#         cdf_qhat = np.quantile(
#             conformal_demo.cdf_naive_ordinal_score_function(
#                 np.copy(cal_scores),
#                 np.copy(cal_labels)
#             ),
#             np.ceil((cal_scores.shape[0] + 1)*(1 - alpha))/cal_scores.shape[0],
#             interpolation='higher',
#         )
#         cdf_pred_sets = conformal_demo.cdf_naive_ordinal_prediction(np.copy(val_scores), cdf_qhat)
#         cdf_overall_coverage.append(cdf_pred_sets[np.arange(val_scores.shape[0]), val_labels].sum() / val_labels.shape[0])
#         cdf_overall_size.append(cdf_pred_sets.sum(1).mean())
        
#         for _class, cond_res in class_conditional_eval(cdf_pred_sets, val_labels).items():
#             cdf_class_cond_count[_class].append(cond_res['count'])
#             cdf_class_cond_coverage[_class].append(cond_res['coverage'])
#             cdf_class_cond_size[_class].append(cond_res['size'])
#         for _size, cond_res in size_conditional_eval(cdf_pred_sets, val_labels).items():
#             cdf_size_cond_count[_size].append(cond_res['count'])
#             cdf_size_cond_coverage[_size].append(cond_res['coverage'])
            
        # Ordinal APS
        aps_qhat = conformal_demo.get_qhat_ordinal_aps(
            conformal_demo.ordinal_aps_prediction,
            np.copy(cal_scores),
            np.copy(cal_labels),
            alpha
        )
        aps_pred_sets = conformal_demo.ordinal_aps_prediction(np.copy(val_scores), aps_qhat)
        aps_overall_coverage.append(aps_pred_sets[np.arange(val_scores.shape[0]), val_labels].sum() / val_labels.shape[0])
        aps_overall_size.append(aps_pred_sets.sum(1).mean())
        for _class, cond_res in class_conditional_eval(aps_pred_sets, val_labels).items():
            aps_class_cond_count[_class].append(cond_res['count'])
            aps_class_cond_coverage[_class].append(cond_res['coverage'])
            aps_class_cond_size[_class].append(cond_res['size'])
        for _size, cond_res in size_conditional_eval(aps_pred_sets, val_labels).items():
            aps_size_cond_count[_size].append(cond_res['count'])
            aps_size_cond_coverage[_size].append(cond_res['coverage'])

#     alpha_lac_overall_coverage[alpha] = lac_overall_coverage
#     alpha_cdf_overall_coverage[alpha] = cdf_overall_coverage
    alpha_aps_overall_coverage[alpha] = aps_overall_coverage
#     alpha_lac_overall_size[alpha] = lac_overall_size
#     alpha_cdf_overall_size[alpha] = cdf_overall_size
    alpha_aps_overall_size[alpha] = aps_overall_size
        
#     alpha_lac_class_cond_count[alpha] = lac_class_cond_count
#     alpha_cdf_class_cond_count[alpha] = cdf_class_cond_count
    alpha_aps_class_cond_count[alpha] = aps_class_cond_count
#     alpha_lac_class_cond_coverage[alpha] = lac_class_cond_coverage
#     alpha_cdf_class_cond_coverage[alpha] = cdf_class_cond_coverage
    alpha_aps_class_cond_coverage[alpha] = aps_class_cond_coverage
#     alpha_lac_class_cond_size[alpha] = lac_class_cond_size
#     alpha_cdf_class_cond_size[alpha] = cdf_class_cond_size
    alpha_aps_class_cond_size[alpha] = aps_class_cond_size
#     alpha_lac_size_cond_count[alpha] = lac_size_cond_count
#     alpha_cdf_size_cond_count[alpha] = cdf_size_cond_count
    alpha_aps_size_cond_count[alpha] = aps_size_cond_count
#     alpha_lac_size_cond_coverage[alpha] = lac_size_cond_coverage
#     alpha_cdf_size_cond_coverage[alpha] = cdf_size_cond_coverage
    alpha_aps_size_cond_coverage[alpha] = aps_size_cond_coverage

In [551]:
alpha

0.2

In [552]:
conformal_demo.get_qhat_ordinal_aps(
            conformal_demo.ordinal_aps_prediction,
            np.copy(cal_scores),
            np.copy(cal_labels),
            0.3
        )

0.9312292229222922

In [553]:
cal_scores

array([[0.35331951, 0.39139019, 0.1376992 , 0.1175911 ],
       [0.40704626, 0.30213158, 0.24611872, 0.04470345],
       [0.3230599 , 0.22845381, 0.39870435, 0.04978194],
       [0.24423044, 0.10174576, 0.62921612, 0.02480769],
       [0.20869141, 0.30146966, 0.04317189, 0.44666705],
       [0.18338341, 0.28462491, 0.0942735 , 0.43771819],
       [0.23477978, 0.19493143, 0.51256878, 0.05772002],
       [0.40797884, 0.23379133, 0.29543931, 0.06279052],
       [0.18201693, 0.24791572, 0.01579958, 0.55426777],
       [0.28767082, 0.32784006, 0.13424571, 0.25024341],
       [0.3240368 , 0.23130406, 0.33938664, 0.1052725 ],
       [0.30197412, 0.12849947, 0.55130394, 0.01822247],
       [0.22096642, 0.13172165, 0.59506314, 0.0522488 ],
       [0.09317637, 0.02938352, 0.87042247, 0.00701765],
       [0.42105891, 0.34511433, 0.15543957, 0.07838719],
       [0.27859671, 0.30916849, 0.34453004, 0.06770477],
       [0.16870951, 0.05845742, 0.75951062, 0.01332244],
       [0.29761451, 0.25686205,

In [554]:
# marginal coverages

aps_agg_coverage = np.array(list(alpha_aps_overall_coverage.values()))
# lac_agg_coverage = np.array(list(alpha_lac_overall_coverage.values()))
# cdf_agg_coverage = np.array(list(alpha_cdf_overall_coverage.values()))

aps_agg_size = np.array(list(alpha_aps_overall_size.values()))
# lac_agg_size = np.array(list(alpha_lac_overall_size.values()))
# cdf_agg_size = np.array(list(alpha_cdf_overall_size.values()))

In [555]:
agg_aps_cvg = []
for x in aps_agg_coverage:
    agg_aps_cvg.append(x.mean()) 
agg_aps_cvg

[1.0,
 0.9728947368421054,
 0.9122631578947369,
 0.8759999999999998,
 0.8166842105263159]

In [556]:
agg_aps_size = []
for x in aps_agg_size:
    agg_aps_size.append(x.mean()) 
agg_aps_size

[4.0,
 3.972684210526316,
 3.910157894736842,
 3.8655263157894737,
 3.7798421052631577]

In [181]:
aps_sev_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['sev'] for alpha in alphas])
aps_mod_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['mod'] for alpha in alphas])
aps_mil_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['mil'] for alpha in alphas])
aps_neg_coverage = np.array([alpha_aps_class_cond_coverage[alpha]['neg'] for alpha in alphas])

In [182]:
aps_neg_cvg = []
for x in aps_neg_coverage:
    aps_neg_cvg.append(x.mean())
aps_neg_cvg

[0.9647882288776549,
 0.8680238620203284,
 0.8156585198611234,
 0.8016426437462786,
 0.7767824088469394]

In [183]:
aps_mil_cvg = []
for x in aps_mil_coverage:
    aps_mil_cvg.append(x.mean())
aps_mil_cvg

[1.0,
 0.9908977636167537,
 0.9754775590574657,
 0.9579817832365757,
 0.9079794186320386]

In [184]:
aps_mod_cvg = []
for x in aps_mod_coverage:
    aps_mod_cvg.append(x.mean())
aps_mod_cvg

[1.0,
 0.9835290187758171,
 0.8805336045904183,
 0.7750512774143183,
 0.6795112761127017]

In [185]:
aps_sev_cvg = []
for x in aps_sev_coverage:
    aps_sev_cvg.append(x.mean())
aps_sev_cvg

[0.9998512050543983,
 0.9866820721950782,
 0.945816662874484,
 0.894774539596991,
 0.8325281771908765]

In [187]:
severe_conditional_cvg = pd.DataFrame(aps_sev_coverage)
mild_conditional_cvg = pd.DataFrame(aps_mil_coverage)
moderate_conditional_cvg = pd.DataFrame(aps_mod_coverage)
negative_conditional_cvg = pd.DataFrame(aps_neg_coverage)

In [188]:
severe_conditional_cvg.to_csv('/Users/subhrasishchakraborty/Desktop/severe_conditional_cvg.csv')
mild_conditional_cvg.to_csv('/Users/subhrasishchakraborty/Desktop/mild_conditional_cvg.csv')
moderate_conditional_cvg.to_csv('/Users/subhrasishchakraborty/Desktop/moderate_conditional_cvg.csv')
negative_conditional_cvg.to_csv('/Users/subhrasishchakraborty/Desktop/negative_conditional_cvg.csv')